<a href="https://colab.research.google.com/github/alim98/MPI/blob/main/VGG/MPI_Vgg_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essential downloads

In [1]:
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"

!unzip -q downloaded_file.zip

--2025-01-04 16:56:37--  https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.96.132, 2a00:1450:4013:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.96.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264688649 (1.2G) [application/octet-stream]
Saving to: ‘downloaded_file.zip’

downloaded_file.zip 100%[===================>]   1.18G  77.9MB/s    in 17s     

2025-01-04 16:56:56 (71.1 MB/s) - ‘downloaded_file.zip’ saved [1264688649/1264688649]



In [2]:

!pip install transformers scikit-learn matplotlib seaborn torch torchvision umap-learn git+https://github.com/funkelab/funlib.learn.torch.git
!pip install openpyxl


  Cloning https://github.com/funkelab/funlib.learn.torch.git to /tmp/pip-req-build-c4zkk3sg
  Running command git clone --filter=blob:none --quiet https://github.com/funkelab/funlib.learn.torch.git /tmp/pip-req-build-c4zkk3sg
  Resolved https://github.com/funkelab/funlib.learn.torch.git to commit 049729151c7a2c0320a446dc9d3244ac830f7ea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.0 MB/s eta 0:00:00
  Created wheel for funlib.learn.torch: filename=funlib.learn.torch-0.1.0-py3-none-any.whl size=13996 sha256=af80411d6f0f65f755da136afb65f6c6e40e55ff472f644853eba8cf9074cb75
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zx96q9e/wheels/28/91/33/f1ad4734bedb93018d3b9e42c487752ac659d5f05afb697203
Successfully built funlib.learn.torch


In [3]:

import os
import glob
import imageio.v2 as iio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from torch.utils.data import Dataset, DataLoader
from transformers import ViTImageProcessor, ViTModel
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from umap import UMAP
import torch.nn.functional as F

# Vit video

# edited

In [18]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import imageio.v2 as iio
from PIL import Image

from funlib.learn.torch.models import Vgg3D  # Import Vgg3D

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Vgg3DFeatureExtractor class
class Vgg3DFeatureExtractor(Vgg3D):
    def __init__(self, *args, **kwargs):
        super(Vgg3DFeatureExtractor, self).__init__(*args, **kwargs)
        # Remove the classifier by replacing it with an identity layer
        self.classifier = nn.Identity()
        # Add a Flatten layer to convert multi-dimensional features to 2D
        self.flatten = nn.Flatten(start_dim=1)

    def forward(self, x):
        # Pass input through all feature layers
        x = self.features(x)
        # Apply pooling if present
        if hasattr(self, 'avgpool'):
            x = self.avgpool(x)
        x = self.flatten(x)
        # Pass through the (now identity) classifier
        x = self.classifier(x)
        return x

# Define normalization function
def normalize_subvol(subvol):
    """
    Normalize the subvolume to have zero mean and unit variance.
    """
    mean = subvol.mean()
    std = subvol.std()
    if std == 0:
        std = 1
    return (subvol - mean) / std

# Paths and directories
checkpoint_url = "https://dl.dropboxusercontent.com/scl/fo/mfejaomhu43aa6oqs6zsf/AKMAAgT7OrUtruR0AQXZBy0/hemibrain_production.checkpoint.20220225?rlkey=6cmwxdvehy4ylztvsbgkfnrfc&dl=0"
checkpoint_path = 'hemibrain_production.checkpoint'

# Download the checkpoint if it doesn't exist
if not os.path.exists(checkpoint_path):
    os.system(f"wget -O {checkpoint_path} '{checkpoint_url}'")
    print("Downloaded VGG3D checkpoint.")
else:
    print("VGG3D checkpoint already exists.")

model_vgg = Vgg3DFeatureExtractor(
    input_size=(80, 80, 80),      # Only spatial dimensions
    fmaps=24,
    downsample_factors=[(2, 2, 2)] * 4,
    fmap_inc=[2, 2, 2, 2],
    n_convolutions=[4, 2, 2, 2],
    output_classes=7,             # Irrelevant but kept for consistency
    input_fmaps=1                # Number of input channels in checkpoint
)

# Load the pre-trained weights
checkpoint = torch.load(checkpoint_path, map_location='cpu')
state_dict = checkpoint['model_state_dict']

# Remove the classifier weights if present
state_dict = {k: v for k, v in state_dict.items() if not k.startswith('classifier')}

model_vgg.load_state_dict(state_dict, strict=False)
model_vgg.to(device)
model_vgg.eval()
print("Vgg3DFeatureExtractor model loaded with 1 input channel and set to evaluation mode.")

first_conv = model_vgg.features[0]
if isinstance(first_conv, nn.Conv3d):
    new_conv = nn.Conv3d(
        in_channels=3,
        out_channels=first_conv.out_channels,
        kernel_size=first_conv.kernel_size,
        stride=first_conv.stride,
        padding=first_conv.padding,
        dilation=first_conv.dilation,
        bias=(first_conv.bias is not None)
    )

    new_conv.weight.data = first_conv.weight.data.repeat(1, 3, 1, 1, 1) / 3

    if first_conv.bias is not None:
        new_conv.bias.data = first_conv.bias.data

    model_vgg.features[0] = new_conv
    print("Modified the first convolutional layer to accept 3 input channels.")
else:
    raise TypeError("Expected the first layer to be nn.Conv3d")

model_vgg.to(device)
model_vgg.eval()

class SynapseDataset(Dataset):
    """
    Loads each synapse from synapse_df, extracts an 80^3 subvolume around central_coord,
    merges raw intensity + side1/side2 masks into a 3-channel volume ( shape => [3, 80, 80, 80] ).
    """
    def __init__(self, vol_data_list, synapse_df, subvol_size=80):
        self.vol_data_list = vol_data_list
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.subvol_size = subvol_size
        self.half_size = subvol_size // 2

    def __len__(self):
        return len(self.synapse_df)

    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_index = syn_info['bbox_index']  # which volume to use
        raw_vol, seg_vol = self.vol_data_list[bbox_index]

        # Coordinates
        central_coord = (
            int(syn_info['central_coord_1']),
            int(syn_info['central_coord_2']),
            int(syn_info['central_coord_3'])
        )
        side1_coord = (
            int(syn_info['side_1_coord_1']),
            int(syn_info['side_1_coord_2']),
            int(syn_info['side_1_coord_3'])
        )
        side2_coord = (
            int(syn_info['side_2_coord_1']),
            int(syn_info['side_2_coord_2']),
            int(syn_info['side_2_coord_3'])
        )

        # Masks for side 1 and side 2
        mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)

        # Determine subvolume bounds
        cx, cy, cz = central_coord
        x_start = max(cx - self.half_size, 0)
        x_end   = min(cx + self.half_size, raw_vol.shape[2])
        y_start = max(cy - self.half_size, 0)
        y_end   = min(cy + self.half_size, raw_vol.shape[1])
        z_start = max(cz - self.half_size, 0)
        z_end   = min(cz + self.half_size, raw_vol.shape[0])

        sub_raw    = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
        sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
        sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]

        # Pad each to (80,80,80) if near edges
        desired_shape = (self.subvol_size, self.subvol_size, self.subvol_size)
        dz, dy, dx = sub_raw.shape

        padded_sub_raw    = np.zeros(desired_shape, dtype=sub_raw.dtype)
        padded_sub_mask1  = np.zeros(desired_shape, dtype=np.uint8)
        padded_sub_mask2  = np.zeros(desired_shape, dtype=np.uint8)

        padded_sub_raw[:dz, :dy, :dx] = sub_raw
        padded_sub_mask1[:dz, :dy, :dx] = sub_mask_1
        padded_sub_mask2[:dz, :dy, :dx] = sub_mask_2

        # Now shape => (3, 80, 80, 80)
        # Channels: 0 = side1 mask, 1 = raw intensities, 2 = side2 mask
        sub_3d = np.stack([padded_sub_mask1, padded_sub_raw, padded_sub_mask2], axis=0)

        # Normalize the subvolume
        sub_3d = normalize_subvol(sub_3d)

        # Convert to float tensor
        sub_3d_tensor = torch.from_numpy(sub_3d).float()

        # Convert syn_info row to dict for convenience
        syn_info_dict = syn_info.to_dict()

        return sub_3d_tensor, syn_info_dict

# Define the feature extraction function
def get_vgg3d_features(subvol_batch):
    """
    Extract features using the Vgg3DFeatureExtractor model.

    Args:
        subvol_batch (torch.Tensor): Tensor of shape [B, 3, 80, 80, 80]

    Returns:
        np.ndarray: Array of shape [B, feature_size]
    """
    subvol_batch = subvol_batch.to(device)  # Move to device

    with torch.no_grad():
        # Forward pass through Vgg3DFeatureExtractor
        features = model_vgg(subvol_batch)  # Shape: [B, feature_size]

    features_np = features.cpu().numpy()
    return features_np

# Define the segment mask creation function
def create_segment_masks(seg_vol, side1_coord, side2_coord):
    """
    Returns boolean masks for side1 and side2 in seg_vol (Z, Y, X).
    """
    x1, y1, z1 = [int(c) for c in side1_coord]
    x2, y2, z2 = [int(c) for c in side2_coord]

    seg_id_1 = seg_vol[z1, y1, x1]
    seg_id_2 = seg_vol[z2, y2, x2]

    mask_1 = (seg_vol == seg_id_1) if seg_id_1 != 0 else np.zeros_like(seg_vol, dtype=bool)
    mask_2 = (seg_vol == seg_id_2) if seg_id_2 != 0 else np.zeros_like(seg_vol, dtype=bool)
    return mask_1, mask_2

# Define the list of bounding box names
bbox_names = [f'bbox{i}' for i in range(1,8)]  # e.g., "bbox1", "bbox2", etc.

# Ensure the output directory exists
os.makedirs('csv_outputs', exist_ok=True)

# Initialize the list to store CSV paths
all_csv_paths = []

# Processing Loop
for bbox_name in bbox_names:
    print(f"Processing {bbox_name}...")
    raw_vol, seg_vol = load_bbox_data(bbox_name)
    excel_file = f'/content/{bbox_name}.xlsx'
    syn_df = pd.read_excel(excel_file)

    syn_df['bbox_index'] = 0
    syn_df['bbox_name']  = bbox_name

    vol_data_list = [(raw_vol, seg_vol)]
    dataset_bbox = SynapseDataset(vol_data_list, syn_df, subvol_size=80)
    dataloader_bbox = DataLoader(dataset_bbox, batch_size=2, shuffle=False, num_workers=2)

    bbox_features = []
    bbox_syn_info = []

    for batch_idx, (subvol_batch, syn_infos) in enumerate(dataloader_bbox):
        # subvol_batch: [B, 3, 80, 80, 80]
        feats = get_vgg3d_features(subvol_batch)  # => shape (B, feature_size)
        bbox_features.append(feats)

        syn_infos_df = pd.DataFrame(syn_infos)  # convert list-of-dict to DataFrame
        bbox_syn_info.append(syn_infos_df)

    # Concatenate features and synapse info
    bbox_features = np.concatenate(bbox_features, axis=0)  # shape [N, feature_size]
    bbox_syn_info = pd.concat(bbox_syn_info, axis=0).reset_index(drop=True)

    feature_cols = [f'feat_{j}' for j in range(bbox_features.shape[1])]
    features_df = pd.DataFrame(bbox_features, columns=feature_cols)

    output_df = pd.concat([bbox_syn_info, features_df], axis=1)

    # Write CSV
    output_csv_name = f'csv_outputs/{bbox_name}_features.csv'
    output_df.to_csv(output_csv_name, index=False)
    all_csv_paths.append(output_csv_name)
    print(f"Saved features for {bbox_name} -> {output_csv_name}")

# Merge all individual CSVs into one DataFrame
merged_df = pd.concat([pd.read_csv(p) for p in all_csv_paths], ignore_index=True)
print(f"\nMerged {len(all_csv_paths)} CSVs into one DataFrame with {len(merged_df)} rows.")

# Save the merged DataFrame to a final CSV
merged_csv = 'csv_outputs/all_features_merged.csv'
merged_df.to_csv(merged_csv, index=False)
print(f"Final merged CSV: {merged_csv}")


VGG3D checkpoint already exists.


<ipython-input-18-c8956d741b31>:71: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Vgg3DFeatureExtractor model loaded with 1 input channel and set to evaluation mode.
Modified the first convolutional layer to accept 3 input channels.
Processing bbox1...
Saved features for bbox1 -> csv_outputs/bbox1_features.csv
Processing bbox2...
Saved features for bbox2 -> csv_outputs/bbox2_features.csv
Processing bbox3...
Saved features for bbox3 -> csv_outputs/bbox3_features.csv
Processing bbox4...
Saved features for bbox4 -> csv_outputs/bbox4_features.csv
Processing bbox5...
Saved features for bbox5 -> csv_outputs/bbox5_features.csv
Processing bbox6...
Saved features for bbox6 -> csv_outputs/bbox6_features.csv
Processing bbox7...
Saved features for bbox7 -> csv_outputs/bbox7_features.csv

Merged 7 CSVs into one DataFrame with 509 rows.
Final merged CSV: csv_outputs/all_features_merged.csv


# Dim Red and Visualize

In [23]:
!dir

bbox1.xlsx  bbox6.xlsx			     __MACOSX	      vit_umap_x_vs_y.html
bbox2.xlsx  bbox7.xlsx			     raw	      vit_umap_x_vs_z.html
bbox3.xlsx  csv_outputs			     sample_data      vit_umap_y_vs_z.html
bbox4.xlsx  downloaded_file.zip		     seg
bbox5.xlsx  hemibrain_production.checkpoint  vit_umap3d.html


In [24]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
import umap.umap_ as umap
import plotly.express as px

merged_csv = 'csv_outputs/all_features_merged.csv'
df = pd.read_csv(merged_csv)

feat_cols = [c for c in df.columns if c.startswith('feat_')]
X = df[feat_cols].values  # shape: [N, hidden_size] (e.g. [N, 768])

pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X)  # shape => [N, 50]

# 4) UMAP from 50 -> 3 dims (for 3D visualization)
umap_3d = umap.UMAP(
    n_components=3,
    n_neighbors=15,     # can tune
    min_dist=0.1,       # can tune
    random_state=42
)
X_umap3 = umap_3d.fit_transform(X_pca)  # shape => [N, 3]

# 5) Add UMAP coordinates back to the DataFrame
df['umap_x'] = X_umap3[:,0]
df['umap_y'] = X_umap3[:,1]
df['umap_z'] = X_umap3[:,2]

fig = px.scatter_3d(
    df,
    x='umap_x',
    y='umap_y',
    z='umap_z',
    color='bbox_name',
    hover_data=['central_coord_1', 'central_coord_2', 'central_coord_3']
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)
fig.write_html("VGG_umap3d.html")

fig.show()


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [25]:
import plotly.express as px

# 1) UMAP (x vs. y)
fig_xy = px.scatter(
    df,
    x="umap_x",
    y="umap_y",
    color="bbox_name",  # color by bbox_name => discrete legend
    title="UMAP (x vs y)",
    hover_data=["umap_x", "umap_y", "bbox_name", "Var1"]
)
fig_xy.write_html("VGG_umap_x_vs_y.html")

fig_xy.show()

# 2) UMAP (x vs. z)
fig_xz = px.scatter(
    df,
    x="umap_x",
    y="umap_z",
    color="bbox_name",
    title="UMAP (x vs z)",
    hover_data=["umap_x", "umap_z", "bbox_name", "Var1"]
)
fig_xz.write_html("VGG_umap_x_vs_z.html")

fig_xz.show()

# 3) UMAP (y vs. z)
fig_yz = px.scatter(
    df,
    x="umap_y",
    y="umap_z",
    color="bbox_name",
    title="UMAP (y vs z)",
    hover_data=["umap_y", "umap_z", "bbox_name", "Var1"]
)
fig_xz.write_html("VGG_umap_y_vs_z.html")

fig_yz.show()


In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "UMAP (x vs y)",
        "UMAP (x vs z)",
        "UMAP (y vs z)"
    ]
)


cat_codes = df["bbox_name"].astype("category").cat.codes
trace_xy = go.Scatter(
    x=df["umap_x"],
    y=df["umap_y"],
    mode="markers",
    name="(x vs y)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=True,
        size=5
    ),
    text=df["bbox_name"],    # hover text
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_x=%{x}<br>umap_y=%{y}<extra></extra>"
)
fig.add_trace(trace_xy, row=1, col=1)

trace_xz = go.Scatter(
    x=df["umap_x"],
    y=df["umap_z"],
    mode="markers",
    name="(x vs z)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=False,  # we already have a colorbar in the first subplot
        size=5
    ),
    text=df["bbox_name"],
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_x=%{x}<br>umap_z=%{y}<extra></extra>"
)
fig.add_trace(trace_xz, row=1, col=2)

# 3) UMAP_y vs UMAP_z
trace_yz = go.Scatter(
    x=df["umap_y"],
    y=df["umap_z"],
    mode="markers",
    name="(y vs z)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=False,
        size=5
    ),
    text=df["bbox_name"],
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_y=%{x}<br>umap_z=%{y}<extra></extra>"
)
fig.add_trace(trace_yz, row=1, col=3)

# Adjust layout
fig.update_layout(
    title="2D UMAP Projections (All Pairwise Components)",
    width=1800,   # wide figure
    height=600,
    showlegend=False
)

fig.show()
